In [3]:
from mlxtend.data import loadlocal_mnist
import platform
from sklearn.model_selection import KFold,cross_val_score
from sklearn.svm import SVC

In [2]:
import os
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Matplotlib is building the font cache; this may take a moment.


In [4]:

X, y = loadlocal_mnist(
            images_path=r'H:\MTech_NIT Nagpur\Sem 2\Deployment Of ML models\Assignment\Assignment 2\train-images-idx3-ubyte\train-images.idx3-ubyte', 
            labels_path=r'H:\MTech_NIT Nagpur\Sem 2\Deployment Of ML models\Assignment\Assignment 2\train-labels-idx1-ubyte\train-labels.idx1-ubyte')


In [5]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [6]:
y

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [7]:
X.shape

(60000, 784)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.20,
                                                random_state=10)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

In [12]:
mlflow.set_experiment("MNIST_SVM_Tuning")

<Experiment: artifact_location='file:///h:/MTech_NIT%20Nagpur/Sem%202/Deployment%20Of%20ML%20models/Assignment/Assignment%202/mlruns/569460480170000306', creation_time=1740428155615, experiment_id='569460480170000306', last_update_time=1740428155615, lifecycle_stage='active', name='MNIST_SVM_Tuning', tags={}>

In [ ]:
with mlflow.start_run():
    # Log parameter grid
    mlflow.log_param("param_grid", str(param_grid))

    # Perform hyperparameter tuning using GridSearchCV
    grid_search = GridSearchCV(SVC(), param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Retrieve and log best parameters
    best_params = grid_search.best_params_
    mlflow.log_params(best_params)

    # Save full GridSearch results as CSV
    results_df = pd.DataFrame(grid_search.cv_results_)
    results_csv = "grid_search_results.csv"
    results_df.to_csv(results_csv, index=False)
    mlflow.log_artifact(results_csv)

    # Log the best model
    best_model = grid_search.best_estimator_
    mlflow.sklearn.log_model(best_model, "SVM_Best_Model")

print(f"Best Parameters: {best_params}")

Fitting 3 folds for each of 18 candidates, totalling 54 fits
